# 1. Install and Import Dependencies

In [ ]:
!pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113
    

In [ ]:
!pip install stable-baselines3[extra] protobuf==3.20.*


In [ ]:
!pip install mss pydirectinput pytesseract

In [ ]:
from mss import mss
import pydirectinput
import cv2
import numpy as np
import pytesseract
from matplotlib import pyplot as plt
import time
from gym import Env
from gym.spaces import Box, Discrete

# 2. Build the Environment

## 2.1 Create Environment

In [ ]:
class WebGame(Env):
    def __init__(self):
        super().__init__()
        #setup spaces
        self.observation_space = Box(low=0, high=255, shape=(1,83,100), dtype=np.uint8)
        self.action_space = Discrete(3)
        #capture game frames
        self.cap = mss()
        self.game_location = {'top': 300, 'left': 0, 'width': 600, 'height': 500}
        self.done_location = {'top': 405, 'left': 630, 'width': 660, 'height': 70}
        
    #actions that we wanna play
    def step(self, action):
        action_map = {
            0:'space',
            1: 'down', 
            2: 'no_op'
        }
        if action !=2:
            pydirectinput.press(action_map[action])

        done, done_cap = self.get_done() 
        observation = self.get_observation()
        reward = 1 
        info = {}
        return observation, reward, done, info
        
    #restart the game
    def reset(self):
        time.sleep(1)
        pydirectinput.click(x=1000, y=600)
        pydirectinput.press('space')
        return self.get_observation()
    
    #visualize the game
    def render(self):
        cv2.imshow('Game', self.current_frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            self.close()
            
    #this closes the observation
    def close(self):
        cv2.destroyAllWindows()
    
    #get the part that we want from the game
    def get_observation(self):
        raw = np.array(self.cap.grab(self.game_location))[:,:,:3].astype(np.uint8)
        gray = cv2.cvtColor(raw, cv2.COLOR_BGR2GRAY)
        resized = cv2.resize(gray, (100,100))
        channel = np.reshape(resized, (1,100,100))
        return channel
    
    #get the game over text and validate it to restart the game
    def get_done(self):
        done_cap = np.array(self.cap.grab(self.done_location))
        done_strings = ['GAME', 'GAHE','SAME']
        done=False
        done = False
        res = pytesseract.image_to_string(done_cap)[:4]
        if res in done_strings:
            done = True
        return done, done_cap

## 2.2 Test Environment


In [ ]:
env = WebGame()


In [ ]:
obs=env.get_observation()


In [ ]:
plt.imshow(cv2.cvtColor(obs[0], cv2.COLOR_GRAY2BGR))


In [ ]:
pytesseract.pytesseract.tesseract_cmd='C:\Program Files\Tesseract-OCR\Tesseract.exe'


In [ ]:
done, done_cap = env.get_done()


In [ ]:
plt.imshow(done_cap)


In [ ]:
pytesseract.image_to_string(done_cap)[:4]


In [ ]:
env.reset() 

In [ ]:
for episode in range(10): 
    obs = env.reset()
    done = False  
    total_reward   = 0
    while not done:  
        obs, reward,  done, info =  env.step(env.action_space.sample())
        total_reward  += reward
    print('Total Reward for episode {} is {}'.format(episode, total_reward))    

# 3. Train the Model


## 3.1 Create Callback

In [ ]:
#import os for file path management
import os 
#import Base Callback for saving models
from stable_baselines3.common.callbacks import BaseCallback
#check Environment    
from stable_baselines3.common import env_checker

In [ ]:
env_checker.check_env(env)


In [ ]:
class TrainAndLoggingCallback(BaseCallback):

    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)

        return True

In [ ]:
CHECKPOINT_DIR = './train/'
LOG_DIR = './logs/'

In [ ]:
callback = TrainAndLoggingCallback(check_freq=10000, save_path=CHECKPOINT_DIR)


## 3.2 Build DQN and Train


In [ ]:
from stable_baselines3 import DQN
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv, VecFrameStack

In [ ]:
env = WebGame()


In [ ]:
model = DQN('CnnPolicy', env, tensorboard_log=LOG_DIR, verbose=1, buffer_size=200000, learning_starts=1000)


In [ ]:
model.learn(total_timesteps=20000, callback=callback)


# 4. Test out Model


In [ ]:
for episode in range(5): 
    obs = env.reset()
    done = False
    total_reward = 0 ion, _ = model.predict(obs) 
        obs, reward, done, info = env.step(int(action))
        time.sleep(0.01)
        total_reward += reward
    print('Total Reward for episode {} is {}'.format(episode, total_reward))
    time.sleep(2)